In [43]:
!pip install memory_profiler
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [44]:
import pandas as pd, os, csv, time, random
from collections import defaultdict

# seed acak setiap run
RANDOM_SEED = int(time.time()) % 100000
random.seed(RANDOM_SEED)
print("🎲 Random seed:", RANDOM_SEED)

CSV_PATH = "/content/sample_data/jadwal_kuliah_random_fix9_halfhour.csv"
N_STUDENTS = 300

REQUIRED = {"EF234501","EF234502","EF234503","EF234504","EF234505"}
ELECTIVES = {"EF234509","EF234518","EF234713","EK234601","EF234618",
             "EF234606","ER234504","EF234614","EF234704","EF234708"}


🎲 Random seed: 90016


In [45]:
def to_minutes(hhmm):
    h, m = map(int, hhmm.split(":"))
    return h * 60 + m

def conflict(a, b):
    return a["day"] == b["day"] and not (a["end"] <= b["start"] or b["end"] <= a["start"])

def load_schedule(path):
    data = {}
    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for r in reader:
            code = r["Kode Kelas"].split()[0]
            cap = 60 if code in REQUIRED else 30
            data[r["Kode Kelas"]] = {
                "code": code,
                "name": r["Nama Mata Kuliah"],
                "day": r["Hari"],
                "start": to_minutes(r["Mulai"]),
                "end": to_minutes(r["Selesai"]),
                "cap": cap,
            }
    return data

def group_by_course(sections):
    g = defaultdict(list)
    for sid, s in sections.items():
        g[s["code"]].append(sid)
    return g


In [46]:
sections = load_schedule(CSV_PATH)
course_to_secs = group_by_course(sections)
capacity = {sid: s["cap"] for sid, s in sections.items()}

print("Total sections:", len(sections))
print("Total kapasitas wajib:", sum(s['cap'] for s in sections.values() if s['code'] in REQUIRED))
print("Total kapasitas pilihan:", sum(s['cap'] for s in sections.values() if s['code'] in ELECTIVES))


Total sections: 45
Total kapasitas wajib: 1500
Total kapasitas pilihan: 600


In [47]:
import time
import random

def to_minutes(hhmm):
    h, m = map(int, hhmm.split(":"))
    return h * 60 + m

def conflict(a, b):
    return a["day"] == b["day"] and not (a["end"] <= b["start"] or b["end"] <= a["start"])

def load_schedule(path):
    data = {}
    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for r in reader:
            code = r["Kode Kelas"].split()[0]
            cap = 60 if code in REQUIRED else 30
            data[r["Kode Kelas"]] = {
                "code": code,
                "name": r["Nama Mata Kuliah"],
                "day": r["Hari"],
                "start": to_minutes(r["Mulai"]),
                "end": to_minutes(r["Selesai"]),
                "cap": cap,
            }
    return data

def group_by_course(sections):
    g = defaultdict(list)
    for sid, s in sections.items():
        g[s["code"]].append(sid)
    return g

def fc_assign_student(course_to_secs, sections, capacity, time_limit=0.3):
    start_time = time.time()

    courses = list(REQUIRED) + random.sample(list(ELECTIVES), 2)
    random.shuffle(courses)
    domains = {c: [sid for sid in course_to_secs[c] if capacity[sid] > 0] for c in courses}
    assigned = {}

    def forward_check(code, sec):
        reduced = {}
        for c2, dom in domains.items():
            if c2 == code or c2 in assigned:
                continue
            new_dom = [s for s in dom if not conflict(sections[s], sections[sec])]
            if new_dom:
                reduced[c2] = new_dom
            else:
                return None
        return reduced

    def backtrack(i=0):
        # stop kalau timeout
        if time.time() - start_time > time_limit:
            return False
        if i == len(courses):
            return True
        c = courses[i]
        dom = sorted(domains[c], key=lambda sid: capacity[sid], reverse=True)
        random.shuffle(dom)
        for sid in dom:
            if capacity[sid] <= 0:
                continue
            assigned[c] = sid
            capacity[sid] -= 1
            reduced = forward_check(c, sid)
            if reduced is not None:
                saved = {k: domains[k][:] for k in reduced}
                for k, v in reduced.items():
                    domains[k] = v
                if backtrack(i + 1):
                    return True
                for k, v in saved.items():
                    domains[k] = v
            capacity[sid] += 1
            assigned.pop(c, None)
        return False

    ok = backtrack(0)
    return assigned if ok else None

def count_conflicts(assignment, sections):
    conflicts = 0
    assigned_sections = [sections[sid] for sid in assignment.values()]
    for i in range(len(assigned_sections)):
        for j in range(i + 1, len(assigned_sections)):
            if conflict(assigned_sections[i], assigned_sections[j]):
                conflicts += 1
    return conflicts

def min_conflicts_assign_student(course_to_secs, sections, capacity, max_steps=1000):
    courses = list(REQUIRED) + random.sample(list(ELECTIVES), 2)
    assigned = {}

    # random assignment awal
    for c in courses:
        available_sections = [sid for sid in course_to_secs[c] if capacity[sid] > 0]
        if not available_sections:
            return None
        assigned[c] = random.choice(available_sections)

    for _ in range(max_steps):
        if count_conflicts(assigned, sections) == 0:
            return assigned

        # cari conflicting course
        conflicting_courses = []
        assigned_sections = list(assigned.values())
        for i, sid1 in enumerate(assigned.values()):
            for j, sid2 in enumerate(assigned.values()):
                if i < j and conflict(sections[sid1], sections[sid2]):
                    conflicting_courses.append(list(assigned.keys())[i])
                    break # tambahkan salah satu
        if not conflicting_courses:
            return assigned

        course_to_fix = random.choice(conflicting_courses)

        # cari best section (minim conflict)
        best_section = None
        min_conf = float('inf')
        available_sections = [sid for sid in course_to_secs[course_to_fix] if capacity[sid] > 0]

        for sec_id in available_sections:
            temp_assigned = assigned.copy()
            temp_assigned[course_to_fix] = sec_id
            conf = count_conflicts(temp_assigned, sections)
            if conf < min_conf:
                min_conf = conf
                best_section = sec_id

        if best_section:
            assigned[course_to_fix] = best_section
        else:
             # If no better section found, try a random one from available
            assigned[course_to_fix] = random.choice(available_sections)


    return None

In [48]:
%%memit

import os
import time
import random

out_dir = "/content/output_CP_optimized"
os.makedirs(out_dir, exist_ok=True)

start = time.time()
success = 0
failed = []

students = [f"S{i:03d}" for i in range(1, N_STUDENTS + 1)]
random.shuffle(students)  # acak urutan mahasiswa biar adil

# Make a copy of initial capacity for each student attempt
initial_capacity = capacity.copy()

for sid in students:
    alloc = None
    current_capacity = initial_capacity.copy() # Use a fresh capacity for each student
    # Try Min-Conflicts first
    alloc = min_conflicts_assign_student(course_to_secs, sections, current_capacity)

    if not alloc:
        # If Min-Conflicts fails, try Forward Checking (can adjust attempts)
        for _ in range(50): # Reduced attempts for FC as MC is primary
            current_capacity = initial_capacity.copy() # Fresh capacity for each FC attempt
            alloc = fc_assign_student(course_to_secs, sections, current_capacity)
            if alloc:
                break


    if not alloc:
        failed.append(sid)
        continue

    # Update the main capacity based on the successful allocation
    for sec_id in alloc.values():
        capacity[sec_id] -= 1

    day_order = {"Senin":1,"Selasa":2,"Rabu":3,"Kamis":4,"Jumat":5}
    rows = []
    for c, sec in alloc.items():
        info = sections[sec]
        jenis = "Wajib" if c in REQUIRED else "Pilihan"
        rows.append([
            info["day"],
            f"{info['start']//60:02d}:{info['start']%60:02d}",
            f"{info['end']//60:02d}:{info['end']%60:02d}",
            sec,
            info["name"],
            jenis,
        ])
    rows.sort(key=lambda r: (day_order[r[0]], r[1]))
    with open(os.path.join(out_dir, f"{sid}.csv"), "w", encoding="utf-8") as f:
        f.write("Hari,Mulai,Selesai,Kode Kelas,Nama Mata Kuliah,Jenis\n")
        for r in rows:
            f.write(",".join(r) + "\n")
    success += 1
    if success % 25 == 0:
        print(f"Progress: {success}/{N_STUDENTS}")

elapsed = time.time() - start
print(f"\n✅ Selesai dalam {elapsed:.2f} detik. Sukses: {success}/{N_STUDENTS}")
if failed:
    print("Mahasiswa gagal:", len(failed), failed[:10])
else:
    print("🔥 Semua mahasiswa berhasil teralokasi!")

Progress: 25/300
Progress: 50/300
Progress: 75/300
Progress: 100/300
Progress: 125/300
Progress: 150/300
Progress: 175/300
Progress: 200/300
Progress: 225/300
Progress: 250/300
Progress: 275/300
Progress: 300/300

✅ Selesai dalam 0.30 detik. Sukses: 300/300
🔥 Semua mahasiswa berhasil teralokasi!
peak memory: 179.34 MiB, increment: 0.00 MiB
